# OverFeat

[OverFeat](https://arxiv.org/pdf/1312.6229.pdf)是纽约大学Yann LeCun实验室在2014年发表的一篇文章,主要贡献了一个使用同一个卷积网络完成了多个任务的方法.算法的思路很简单也很牛逼:输入一个raw的图像,通过CNN共同的某些层,同时完成[classification，localization和detection](https://zhuanlan.zhihu.com/p/31727402)的任务.

另外我们用一句话解释一下什么叫OverFeat,简单的来说就是特征提取算子,共用特征网络的某些层(比如前5层)做不同的任务,以下是paper的原话.
> Along with this paper, we release a feature extractor named “OverFeat”.

那么现在先让我们来看看计算机视觉(Computer Vision)的三大任务:

### Computer Vision

[1] 分类(Classification,是什么):

给定一张图片,为每张图片打一个标签,说出图片是什么物体.然而因为一张图片中往往有多个物体,因此我们允许你取出概率最大的5个,只要前五个概率最大的包含了我们人工标定标签,就认定正确也就是Top-K预测.也就是我们之前一直在做的东西,对于二分类而言可以看做是Top-1的预测.

![](https://pic3.zhimg.com/80/v2-bcdb8d70c678d4ebc58aacf7e871c71b_hd.jpg)
    
[2] 定位(Localization,在哪儿):

除了需要预测出图片的类别,你还要定位出这个物体的位置,同时规定你定位的这个物体框与正确位置差不能超过规定的阈值.需要注意的是,这里的定位是针对于一个对象而言,并且这里说所的阈值在现在一般指的是[IOU](https://blog.csdn.net/u014061630/article/details/82818112).

![](https://pic3.zhimg.com/80/v2-f4efa5dbba5ad986d197530b13d2d716_hd.jpg)


[3] 检测(Detection,在哪儿有什么):给定一张图片,你把图片中的所有物体全部找出来(包括位置、类别),一般这种会使用Fast-RCNN,YOLO等.

![](https://pic4.zhimg.com/80/v2-312129e2d0c41bbb02df8e4ce511bd7d_hd.jpg)

那么OverFeat Net就是想一口气解决这三个问题,那么我们接下来就好好的来观赏一下OverFeat的牛逼详情.

**Ps:**

在本文的代码中并没有涉及到检测的问题.

### 1. Classification 

#### 1.1 AlexNet 图片分类回顾


因为OverFeatNet的架构方面和AlexNet相似,所以我们先来回顾一下AlexNet的训练过程:

1.训练阶段:每张训练图片$256\times256$，然后我们随机裁剪出$224\times224$大小的图片,作为CNN的输入进行训练.


2.测试阶段:输入$256\times256$大小的图片,我们从图片的5个指定的方位(上下左右+中间)进行裁剪出5张$224\times224$大小的图片,然后水平镜像一下再裁剪5张,这样总共有10张;然后我们把这10张裁剪图片分别送入已经训练好的CNN中,分别预测结果,最后用这10个结果的平均作为最后的输出.


另外AlexNet的裁剪方式实际上也是窗口滑动(Window sliding)的一种体现方式,用来寻找一张图片不同地方的特征区别.

需要注意的是,实际上AlexNet架构有一些不足的地方:

- 测试的时候,图片的输入只能是规定的$256\times256$,而不能是任意大小的图片.

- 由于ImageNet中大部分的图片对象都在图片的中心,那么依照5个方位进行裁剪可能问题并不大,但是如果是放在一般情况,那么你的裁剪很有可能将对象裁剪残缺,比如给予一张猫的脸,运气不好的情况就会将猫脸的一半给裁剪丢失,进而导致特征信息不完整.

然而OverFeat对于图片分类算法而言,在训练阶段和AlexNet基本相似,但是在测试阶段差别很大,并是不采用裁剪的方式,而是使用"Multi-Scale"和"Offset".

那么接下来我们来看看OverFeat比较重要的基础知识.

#### 1.2 FCN 全卷积网络

FCN又称全卷积神经网络,是现如今是图片语义分割领域的新宠(Fully Convolutional Networks for Semantic Segmentation),同时也是OverFeat的理论依据.

我们在上面知道,对于AlexNet的测试阶段,输入大小是固定的,那么如果我们输入随意一张大小的图片(比如$500\times 500$)并且保证网络能够一直向前传播.这就是FNC的精髓所在,它能够使得网络接收任意大小的图片.

<img src="../../../picture/230.png" height=800 width=800 />

上图就是FNC的一个简单的示意图.

我们先来看上半部分(train):

1.输入的一个大小为$14\times 14$的图片,经过第一次conv(kernel:$5\times5$,stride:1)后得到$10\times 10$,再经过一个max pool(kernel:$2\times2$,stride:2)后得到$5\times 5$.再这里基本没有什么变化,但是进入FC层(classifier)后就开始变化了,我们使用一个(kernel:$5\times 5$,stride:1)的卷积核进行卷积操作,得到一个$1\times 1$的结果,接着继续使用两次(kernel:$1\times 1$,stride:1)的卷积核进行卷积操作,最后得到输出层的$1\times 1$

其实这里只是将之前的全连接层更改为使用卷积进行操作,仔细回想一下,全连接层实际上说白了就是线性操作,比如讲将我们的**1st stage**的$5\times 5$展平为$25\times 1$的形状,那么我们的weights就应该是$1\times 25$,输出就为$1\times 1$的.然而我们使用一个与上一层输出相同大小的卷积核进行卷积操作,实际上也是一个线性操作.两者的性质是等价的,那么这么做有什么好处呢?我们来看看测试阶段.

下半部分(test):

2.在测试阶段的时候,我们随意输入一张大小的图片,假设为$16\times 16$,那么可以发现网络是能够正常的前向传播的,但是不同的是最后输出的大小不再是$1\times 1$而是$2\times 2$,虽然得出的结果不同,但其实这个$2\times 2$相当于不同输入尺度下的预测值(将$2\times2$展平),换句话说,我们输入$16\times 16$的图,输出的是4个预测值,从感受野的角度来看,最后的$2\times 2$的部分中红色的部分相当于输入图片中红色框起来的部分,最后的输出的蓝色的部分相当于输入图片中蓝色框起来的部分.从这个角度来看,我们实际上在做和"裁剪"一样的事情,但是好处是我们没有因为"裁剪"而丢失图片中某些特征.所以使用FNC后输入不同尺度下图片得到的不同结果进行求平均概率是和AlexNet裁剪后取平均概率的思想是类似的.

所以FNC的灵魂很简单:

1.把卷积层到全连接层看成是一个**卷积操作**.

2.把全连接层到全连接层看成是一个$1\times 1$**卷积核大小的卷积运算**.

看完FNC,我们来看看OverFeatNet的一个小小的创新点-offset(实际上经过后面的研究证实效果不是很明显).

#### 1.3 Offset max-pooling

在OverFeatNet中一个创新点是**Offset**池化,为了方便说明,我们先用一维示例.

![](../../../picture/231.png)

如上图所示,我们在轴上有20个神经元,如果我们选择池化size=3,stride=1的非重叠池化,那么依据我们之前所学的方法应该是:

对上面的20个神经元从1开始进行分组,每三个为一组,然后计算最大值.19,20号神经元抛弃.当然我们也可以人为的在20号神经元后面加上一个数值为0的神经元,那么$[19,20,21]$就可以被分为一组继而计算最大值,那么就是我们之前所学的padding了.

然而我们未必要从1开始进行分组,从2,3进行分组也是可以的,也就是说如果我们只要分6组的话(no padding),我们有三种组合方式:

- $\Delta = 0: [1,2,3],[4,5,6],...,[16,17,18]$

- $\Delta = 1: [2,3,4],[5,6,7],...,[17,18,19]$

- $\Delta = 2: [3,4,5],[6,7,8],...,[18,19,20]$

也就是对应于下面的图片

![](../../../picture/232.png)

以往的CNN中,我们一般只用了$\Delta = 0$的情况,池化后我们就送入下一层.然而OverFeatNet的方法是上面的$\Delta=0,\Delta=1,\Delta=2$的三种结果组合方式的结果分别送入下一层网络.这样的话我们在输出的时候就有3种预测结果了.

那么前面所说的就是一维的情况,由于我们在做池化的时候是针对于二维(height:$\Delta x$,width:$\Delta y$)的,所以($\Delta x$,$\Delta y$)的融合就会产生9中情况(3$\times$3);那么如果我们在做图片分类的时候,使用这种方法,那么就会得到9种结果即:每个类别都有9中概率值,我们对于这9种概率值取最大的那个,作为此类的预测概率值.

#### 1.4 classifier

对于分类问题而言,一个常用的增加训练样本的方法是将训练样本随机移动一个小的位移,或者等价的在图像中随机取一些大的图像块,然后以这些图像块为输入训练分类模型.在测试阶段,可以采用滑窗的方法对每一个图像块进行分类,然后组合这些分类结果,得到一个置信度更高的类别标签.

在OverFeat中利用FNC和offset max-pooling来达到类似的效果.

#### 1.4.1 Architecture



对于架构方面,OverFeat有两个版本,一个是快速版本,一个是精确版本,精确版本速度比较慢(实际上对于现在的硬件而言两者的速度也相差不大).

**Architecture specifics for fast model**

![](../../../picture/233.png)


**Architecture specifics for accurate model**

![](../../../picture/234.png)

**Ps:**

代码采用的是fast model,所以我们说的是针对于fast model.

从上面的表格中我们知道,我们输入的是$231\times 231$的图片,前两层使用的是卷积+池化的操作,特征提取层只到第5层,第6层开始到输出层(第8层)使用全卷积的方式,也是使用ReLu激活函数和最大池化,与AlexNet不同之处在于:

- 1.没有使用局部响应归一(LRN);

- 2.没有采用重叠池化的方法;

- 3.第一层中卷积的步长作者选择的是2(精确版本),而AlexNet选择是4,更小的步长可以获得更高的精度,但是速度会变慢.

**需要注意的是,我们需要将第6层看做是一个kernel:$6\times 6$,stride:$1\times 1$的卷积操作**,另外由于使用FNC,所以在测试的时候我们可以输入任意大小的图片.

#### 1.4.2 Training

对于OverFeatNet的训练有以下内容:

输入: 对于每张图片为$256\times 256$,然后随机裁剪为$231\times 231$的大小作为输入,进行训练.

参数配置: 训练的mini-batch选择是128,weights初始化选择Gaussian分布的随机初始化$(\mu,\sigma)=(0,0.001)$.

优化器: 选择是随机梯度下降法,动量参数大小选择0.6,L2权重衰减系数选择大小为$10^{-5}$.学习率一开始选择0.05,然后每个十几次迭代就将学习率减小为原来的一半.

最后Dropout只在最后两个全连接层使用,比率为0.5

**Ps:**

代码并没有采用以上的方式,对于输入由于硬件和数据集的限制,并没有严格按照该输入模型进行训练,详细参考代码部分.

#### 1.5 Multi-Scale

多尺度预测是OverFeat的最大亮点,简单来说,由于FNC的使用,我们可以输入任意大小的图片使得直接获得多种预测结果,另外加上之前所说的offset就得到了Multi-Scale.

需要注意的是,一旦网络训练完毕了,所有的参数也就确定了,网络结构也确定了.

OverFeat在测试的时候使用了6种不同尺度(scale)的图片作为输入,实际上在原文中是使用12种,另外6种是水平翻转的结果.

以下的表格是针对于**accurate model**的.

![](../../../picture/235.png)

在测试的时候,输入的图片scale分别是$245\times 245,281\times 317,...,461\times 569$.

当网络前向传播到layer 5的时候就采用了之前所说的offsst和FNC的结合,这里我们以$281\times 317$为例(其他的都是一个道理),需要注意的是,我们需要将模型中的**layer 6**看着是上表中的**layer 5**且是针对**accurate model**而言,那么接下来说明layer5 前后的具体流程:

![](../../../picture/236.png)

当($281\times 317$)向前流动到layer5的时候,没有进入max pool之前:

[1] 实现layer5 pre-pool 到 layer5 post-pool的过程实际上是一个offset的过程(图中的a,b).以(3,3)进行池化,然后$\Delta x=0,1,2$,$\Delta y=0,1,2$,这样我们可以得到9种不同的结果也就是表格中scale-2 Layer5 post-pool中后面的($3\times 3$).

[2] 接着分别送入layer7,就可以得到Classifier map(pre-reshape)中的$2\times 3$的大小($\frac{(6-5)}{1} + 1,\frac{(7-5)}{1} + 1$),然后layer8,layer9并不改变形状只是进行线性操作(也就是上图c,d).注意:需要将(6,7,8)看做是(7,8,9),才能够依照表格进行对比.

[3]最后进行reshape(e)后得到$6\times 9\times C$的结果,最后将结果拉伸成为$N\times C$($54\times C$)的形状,我们就可以得到当前scale下N个预测值,我们选取最大的概率值作为预测概率.

这里C指的是类别数量,需要注意的是上图的$\times 256$ maps,不太明白它的意思,因为按照模型架构应该是4096,当然如果作者只是举一个例子的话那就没有任何关系了.

最后每一个尺度都有一个概率值,那么我们就会得到6个最可信的预测概率值,接着我们依照这6个概率值做平均标签预测即可,也就是说,经过测试模型的时候,我们针对于当前6个尺度一共做了$3\times 3 + 6\times 9 + 9\times 15 + 15 \times 21 + 18 \times 24 + 21 \times 30$这么多次预测,那么多次预测出来的值提供了更高的可信度.

到此,OverFeat的分类任务就结束了,总结一下,我们可以将网络看成2个部分:1.特征提取层layer1-layer5;2.分类层:layer6-output.这种思想也被后面许多的模型所学习.

下面我们来看看原文对于ImageNet的训练结果:

![](../../../picture/237.png)

表格2:在验证集上的分类实验结果.Fine/coarse stride 在此处是指offset-max-pooling中∆ 的数量.精细:∆ = 0, 1, 2; 粗糙:∆ = 0.

另外可以看到是否使用offset对模型精度的提高并没有太大作用,比如:

![](../../../picture/238.png)

可以看到top-1 error也就只是降低了0.2%左右,top-5 error也只是降低了1%.所以无论使用何种模型,多种方式结合才是制胜的王道.

### 2. Localization

对于给定一张图片,我们不仅要判断图片所属的类别,我们还要知道物体具体在哪里,这就是定位(Localization)任务.

对于传统的定位任务,一个比较直观解决方案是"滑窗",也就是说通过窗口的滑动来逐一判断这个窗口内是否含有目标.但是这种方法有一个不足之处:窗口需要密集采样,从而导致计算量过大.


<img src="../../../picture/239.png" height=500 width=500 />


在OverFeat中,为了处理定位任务,基于训练的分类网络,用一个**回归网络**替换**分类器网络**(layer 6~output);并在各种缩放比例和view下训练回归网络来预测边框(boundingbox);然后融合预测的各个bounding box.



#### 2.1 Regression

**Train:**

[1] 在OverFeat中将Classification中**训练完毕**的前5层(layer1-layer5)作为特征提取层放在Regression model中.

[2] 将Classification中的layer6-Outlayer替换为回归层,回归层有两层:分别是4098和1024个神经元(作者没有说为什么,应该是交叉验证后得到的),对于回归层也是采用FNC的模式.

[3] 最后连接一个4个神经元的输出层作为bounding box的四个值(xmin,ymin,xmax,ymax).

[4] 使用L2-loss来训练模型.

**Test:**

[1] 图片先经过Classification后得到最可信的top-K分类结果.

[2] 将这K类结果**分别**送入**不同**的回归模型,换句话说,每一个类别都有一个回归模型,比如假设我们有1000个类别,那么就需要训练1000个回归模型.亦即有C个版本的回归模型.

[3] 接着将这些预测出来的结果进行融合.


**Ps:**

代码中我使用的是mean_squared_error而不是论文中使用的L2-Loss.

#### 2.1.1 Offset of Regression

<img src="../../../picture/240.png" height=500 width=500 />

对第5层池化后(也就是accurate model 中的layer6)的特征图应用回归网络,以表格中scale=2的网络为例.

(a)对于每个(3x3) ∆x, ∆y,神经网络的输入是6x7x256,其中6x7是单个特征图的空间像素分布,256为通道数目.

(b)回归网络的第一层中的每个神经元都连接到第5层特征图中对应的5x5邻域以及所有256个通道,在6x7的特征图中移动5x5邻域,最终得到该层中的所有(3x3)∆x, ∆y和4096个通道的2x3空间范围的特征图.

(c)回归网络的第二层有1024个神经元并且和第一层是全连接的.

(d)回归网络的输出层是一个4维向量,代表2x3特征图中每个点对应的边框位置.

<img src="../../../picture/241.png" height=500 width=500 />

上图展示了在单个比例上预测的在各个offset和sliding window下 pooling后,预测的多个bounding box,可以看出所有的bbox就趋向于一个中心,这是一种良好的表现,并且不同位置也能够被检测出来,但是仔细换观察可以发现采用offset某些bbox也是偏离目标中心的,比如最后一张图,而且bbox的冗余也比较严重,很多边框都是重复的,也就是直接导致OverFeat-Regression无法应用于实时检测的原因之一,对于现代Net,一般IOU在0.5左右就是能够接受的结果,也就不需要那么多的预测值.

#### 2.1.2 Multi-Scale of Regression

在使用多尺度预测后,根据不同尺度所产生的结果也不尽相同.

<img src="../../../picture/242.png" height=500 width=500 />

可以看到,Scale的变化也会使得结果发生变化,在尺度较小的时候并没有检测出"鱼"而仅仅只是检测出了"熊",并且不同尺度下的预测bbox偏离程度和精确程度也不一样,Scale比较的小的时候bbox相对比较粗糙,当然这还是相对于训练时候的图片大小以及相应训练label的处理.

#### 2.1.3  Combining Predictions

知道了offset和Multi-Scale后我们需要将结果进行融合,作者给出了下面的算法:

<img src="../../../picture/243.png" height=500 width=500 />

<img src="../../../picture/244.png" height=500 width=500 />

(a) 在6个缩放比例上运行分类网络,在每个比例$s$上选取top-k个类别,作为图片所属类别的集合,设为$Cs$.

(b) 在每个缩放比例的所有空间位置上运行回归网络,产生类别集合$Cs$中每个类别对应的bounding box集合,设为$Bs$.

(c) 各个比例的Bs的并集设为B.

(d) 重复执行以下合并策略直到结束:

(e) 计算B中任意两个边界框b1,b2的匹配分数,找到差异最小的两个边框($b_{1^{*}}, b_{2^{*}}$).

(f) 如果($b_{1^{*}}, b_{2^{*}}$)的差异度 > t,结束合并.

(g) 否则,就在B中删除$b_{1^{*}}, b_{2^{*}}$,然后把$b_{1^{*}}$和$b_{2^{*}}$融合后的边框放入B中.

**Ps:**

不幸的是,我并没有完成这个过程并且对于我的数据集使用单个Scale效果不算太差.如果你有兴趣可以尝试一下.

### Detection

<img src="../../../picture/245.png" height=500 width=500 />

对于定位,论文作者也只是一笔带过了,也就是对于多个窗口同时使用上述的方法从而达到在每个滑动窗口内实现检测,作者也说了这种方式计算量较大.

那么在后面,我们将会使用现在比较流行的方法像FAST-R-CNN,YoLo,SSD等完成Detection.

### Reference 

1.[OverFeat论文阅读笔记](https://zhum.in/blog/project/TrafficSignRecognition/OverFeat%E8%AE%BA%E6%96%87%E9%98%85%E8%AF%BB%E7%AC%94%E8%AE%B0/)

2.[OverFeat 个人总结](http://www.ishenping.com/ArtInfo/251067.html)

3.[OverFeat:用卷积网络同时进行图像识别、定位和检测](http://www.chenzhaobin.com/notes/overfeat)

4.[OverFeat](https://arxiv.org/pdf/1312.6229.pdf)